In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))


# Any results you write to the current directory are saved as output.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline
#import h2o
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator, H2ODeepLearningEstimator
#h2o.init()
import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df
#load json object
#with open('../input/train.json') as f:
 #   d = json.load(f)

#lets put the data into a pandas df
#clicking on raw_nyc_phil.json under "Input Files"
#tells us parent node is 'programs'
#nycphil = json_normalize(d['band_1'])
#nycphil.head(3)
#print(d)
train = pd.read_json("../input/train.json")
test = pd.read_json("../input/test.json")

#h_band_1=h2o.H2OFrame(train["band_1"].values)
#h_band_2=h2o.H2OFrame(train["band_2"].values)
#t_band_1=h2o.H2OFrame(test["band_1"].values)
#t_band_2=h2o.H2OFrame(test["band_2"].values)




In [ ]:
#Generate the training data
#Create 3 bands having HH, HV and avg of both
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

#X_train = np.concatenate([X_band_2[:, :, :, np.newaxis]],axis=-1)
#X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis]],axis=-1)
                          
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [ ]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import SGD

In [ ]:
#MODEL V2
import keras.backend as K
def mean_pred(y_true, y_pred):
    return K.mean(y_pred)
#define our model
def getModel():
    #Building the model
    gmodel = Sequential()
    gmodel.add(Conv2D(20, (3, 3), input_shape=(75, 75,2)))
    gmodel.add(Activation('relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2)))
    gmodel.add(Dropout(0.1))
    
    gmodel.add(Conv2D(40, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2)))
    gmodel.add(Dropout(0.1))    
    
    gmodel.add(Conv2D(20, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2)))
    gmodel.add(Dropout(0.1))    
    
    gmodel.add(Conv2D(20, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2)))
    gmodel.add(Dropout(0.1))    
    
    gmodel.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    gmodel.add(Dense(640))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.5))
    
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))
    
    gmodel.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    gmodel.summary()    
    return gmodel

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=10)

In [ ]:
#MODEL V3
#define our model
def getModelV3():
    #Building the model
    gmodel = Sequential()
    gmodel.add(Conv2D(120, (3, 3), input_shape=(75, 75,3)))
    gmodel.add(Activation('relu'))
    gmodel.add(Conv2D(40, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2)))
   # gmodel.add(Dropout(0.1))
    
    gmodel.add(Conv2D(40, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(Conv2D(40, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2)))
    #gmodel.add(Dropout(0.1))    
    
    gmodel.add(Conv2D(20, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(Conv2D(40, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(Conv2D(40, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2)))
    #gmodel.add(Dropout(0.1))    
    
    gmodel.add(Conv2D(20, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(Conv2D(40, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(Conv2D(40, (3, 3)))
    gmodel.add(Activation('relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2)))
   # gmodel.add(Dropout(0.1))    
    
    gmodel.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    gmodel.add(Dense(640))
    gmodel.add(Activation('relu'))
  #  gmodel.add(Dropout(0.5))
    
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))
    
    gmodel.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    gmodel.summary()    
    return gmodel

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=10)

In [ ]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(75,75,3)))
    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4000, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4000, activation='relu'))
    model.add(Dropout(0.5))
    #model.add(Dense(1000, activation='softmax'))
    model.add(Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer="adamax", loss='binary_crossentropy',metrics=['accuracy'])
    if weights_path:
        model.load_weights(weights_path)

    return model

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

In [ ]:
target_train=train['is_iceberg']
#X_train2 = X_train2.reshape(X_train.shape[0],15, 15,3)
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.99)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
gen_images = ImageDataGenerator(  
  horizontal_flip = True,
  vertical_flip = True,
  width_shift_range = 0.30,
  height_shift_range = 0.30,
  zoom_range = 0.1,
  rotation_range = 20
  )
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
   horizontal_flip = True,
  vertical_flip = True,
  width_shift_range = 0.30,
  height_shift_range = 0.30,
  #zoom_range = 0.1,
  rotation_range = 180
)


test_datagen = ImageDataGenerator(
  horizontal_flip = True,
  vertical_flip = True,
  width_shift_range = 0.30,
  height_shift_range = 0.30,
  #zoom_range = 0.1,
  rotation_range = 180
)


In [ ]:
#Without denoising, core features.
#befor submit this we can try to h2o autoencodes?
import os
gmodel=VGG_16(None)
batch_size=200
train_generator = train_datagen.flow(X_train_cv,y_train_cv,batch_size=batch_size)

validation_generator = test_datagen.flow(X_valid,y_valid,batch_size=batch_size)

# fine-tune the model
gmodel.fit_generator(
        train_generator,
        steps_per_epoch=X_train_cv.shape[0] // batch_size,
        epochs=50,
        verbose=1,
       # validation_data=validation_generator,
         callbacks=get_callbacks(filepath=file_path, patience=5))
        #validation_steps=nb_validation_samples // batch_size)
    
#gmodel.summary()  
#gmodel.fit(X_train_cv, y_train_cv,
 #         batch_size=32,
  #        epochs=50,
   #       verbose=1,
    #      validation_data=(X_valid, y_valid),
     #     callbacks=callbacks)


In [ ]:
print("h")
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
#print('Test loss:', score[0])
print('Test accuracy:', score)

In [ ]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test=gmodel.predict_proba(X_test)

In [ ]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

In [ ]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
#top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu',input_shape=gmodel.output_shape[1:]))
#top_model.add(Flatten(Input(gmodel.output_shape[1:])))
#top_model.add()
#top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning


# add the model on top of the convolutional base
gmodel=gmodel=VGG_16(file_path)
gmodel.load_weights(file_path)
gmodel.add(top_model)





In [ ]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in gmodel.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
gmodel.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:

# featurewise_center = TRUE,
  # featurewise_std_normalization = TRUE,
#gmodel.fit(X_train_cv, y_train_cv,
 #         batch_size=32,
  #        epochs=50,
   #       verbose=1,
    #      validation_data=(X_valid, y_valid),
#          callbacks=callbacks
train_generator = train_datagen.flow(X_train_cv,y_train_cv,batch_size=1000,seed=54321)

validation_generator = test_datagen.flow(X_valid,y_valid,batch_size=400,seed=54321)

# fine-tune the model
gmodel.fit_generator(
        train_generator,
        #steps_per_epoch=nb_train_samples // batch_size,
        epochs=50,
        verbose=1,
        validation_data=validation_generator)
        #validation_steps=nb_validation_samples // batch_size)


#gmodel.fit_generator(
  #flow_images_from_data(x = X_train_cv, y = y_train_cv,generator = gen_images, seed = 123),     
   # epochs = epochs,
   #  verbose=1,
  #     validation_data=(X_valid, y_valid),
 #     callbacks = callbacks
#)

#flow_images_from_data(x = train[[1]], y = train[[3]],generator = gen_images,
 #    batch_size=batch_size, seed = 123)
